<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BERTimbau_Fine_tuning_HAREM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.1 MB/s 
     |████████████████████████████████| 432 kB 70.5 MB/s 
     |████████████████████████████████| 7.6 MB 48.9 MB/s 
     |████████████████████████████████| 163 kB 65.0 MB/s 
     |████████████████████████████████| 212 kB 59.1 MB/s 
     |████████████████████████████████| 115 kB 75.6 MB/s 
     |████████████████████████████████| 127 kB 66.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.7 requires tensorflow>=2.0.0, w

In [2]:
from datasets import load_dataset, load_metric

dataset = load_dataset("harem")
#dataset = load_dataset("wnut_17")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/121 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/128 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset harem downloaded and prepared to /root/.cache/huggingface/datasets/harem/default/1.0.0/36bc40dff7bca67de10e34764fd6795bdf81fc55590b829d4c94437894656dc3. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = AutoModelForTokenClassification.from_pretrained('neuralmind/bert-base-portuguese-cased')

model_checkpoint = "neuralmind/bert-base-portuguese-cased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/210k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [4]:
dataset["train"][0]



{'id': 'HAREM-871-07800',
 'tokens': ['Abraço',
  'Página',
  'Principal',
  'ASSOCIAÇÃO',
  'DE',
  'APOIO',
  'A',
  'PESSOAS',
  'COM',
  'VIH',
  '/',
  'SIDA',
  'A',
  'ABRAÇO',
  'é',
  'uma',
  'Instituição',
  'Particular',
  'de',
  'Solidariedade',
  'Social',
  '.',
  'Organização',
  'não',
  'governamental',
  'sem',
  'fins',
  'lucrativos',
  'de',
  'prestação',
  'de',
  'serviços',
  'na',
  'área',
  'da',
  'SIDA',
  '.',
  'Foi',
  'constituída',
  'por',
  'escritura',
  'pública',
  'em',
  'Junho',
  'de',
  '1992',
  ',',
  'formalizando',
  'e',
  'dando',
  'continuidade',
  'ao',
  'trabalho',
  'de',
  'um',
  'pequeno',
  'número',
  'de',
  'voluntários',
  'que',
  ',',
  'desde',
  'Dezembro',
  'de',
  '1991',
  ',',
  'prestava',
  'apoio',
  'psicológico',
  ',',
  'social',
  'e',
  'material',
  'a',
  'seropositivos',
  'internados',
  'na',
  'Unidade',
  'de',
  'Doenças',
  'Infecciosas',
  'e',
  'Parasitárias',
  'do',
  'Hospital',
  'Egas'

In [5]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-PESSOA',
 'I-PESSOA',
 'B-ORGANIZACAO',
 'I-ORGANIZACAO',
 'B-LOCAL',
 'I-LOCAL',
 'B-TEMPO',
 'I-TEMPO',
 'B-VALOR',
 'I-VALOR',
 'B-ABSTRACCAO',
 'I-ABSTRACCAO',
 'B-ACONTECIMENTO',
 'I-ACONTECIMENTO',
 'B-COISA',
 'I-COISA',
 'B-OBRA',
 'I-OBRA',
 'B-OUTRO',
 'I-OUTRO']

exemplo de entrada

In [7]:
label_all_tokens = True

In [30]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [31]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [32]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--neuralmind--bert-base-portuguese-cased/snapshots/94d69c95f98f7d5b2a8700c420230ae10def0baa/config.json
Model config BertConfig {
  "_name_or_path": "neuralmind/bert-base-portuguese-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "interme

In [33]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
)




PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
example = dataset["train"][0]

metric = load_metric("seqeval")

labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])


{'ABSTRACCAO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'LOCAL': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'ORGANIZACAO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 10},
 'TEMPO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 'VALOR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [43]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [46]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [38]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 121
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 128
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 8
    })
})

In [39]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 121
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 128
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 8
    })
})

In [47]:
#@title Step 13: Pre-training the Model

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags. If tokens, id, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 121
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 320


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.247593,0.699770,0.728647,0.713916,0.949160
2,No log,0.245137,0.708915,0.719290,0.714065,0.949600
3,No log,0.256729,0.712836,0.724808,0.718772,0.950920
4,No log,0.257270,0.707585,0.727447,0.717378,0.950160
5,No log,0.259272,0.714286,0.725768,0.719981,0.950400
6,No log,0.260844,0.700760,0.729846,0.715008,0.949940
7,No log,0.257702,0.706825,0.740403,0.723225,0.951120
8,No log,0.266792,0.722960,0.731286,0.727099,0.951819
9,No log,0.264377,0.710098,0.744002,0.726655,0.951499
10,No log,0.273297,0.714218,0.726727,0.720419,0.950840


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags. If tokens, id, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 128
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags. If tokens, id, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 128
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags. If tokens, id, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Runn

TrainOutput(global_step=320, training_loss=0.008572360873222351, metrics={'train_runtime': 387.8356, 'train_samples_per_second': 6.24, 'train_steps_per_second': 0.825, 'total_flos': 631076771094264.0, 'train_loss': 0.008572360873222351, 'epoch': 20.0})

In [48]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags. If tokens, id, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 128
  Batch size = 8


Perplexity: 1.32


In [49]:
eval_results

{'eval_loss': 0.28017938137054443,
 'eval_precision': 0.7094843462246777,
 'eval_recall': 0.739443378119002,
 'eval_f1': 0.7241541353383458,
 'eval_accuracy': 0.951859256297481,
 'eval_runtime': 5.9279,
 'eval_samples_per_second': 21.593,
 'eval_steps_per_second': 2.699,
 'epoch': 20.0}

In [50]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./BERTimbau-HAREM")

Saving model checkpoint to ./BERTimbau-HAREM
Configuration saved in ./BERTimbau-HAREM/config.json
Model weights saved in ./BERTimbau-HAREM/pytorch_model.bin
tokenizer config file saved in ./BERTimbau-HAREM/tokenizer_config.json
Special tokens file saved in ./BERTimbau-HAREM/special_tokens_map.json


In [51]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline


classifier = pipeline("token-classification", model = "./BERTimbau-HAREM", tokenizer=tokenizer)



loading configuration file ./BERTimbau-HAREM/config.json
Model config BertConfig {
  "_name_or_path": "./BERTimbau-HAREM",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13

In [53]:
classifier("Apple ou Google")



[{'entity': 'LABEL_3',
  'score': 0.4257144,
  'index': 1,
  'word': 'Apple',
  'start': 0,
  'end': 5},
 {'entity': 'LABEL_0',
  'score': 0.8157248,
  'index': 2,
  'word': 'ou',
  'start': 6,
  'end': 8},
 {'entity': 'LABEL_3',
  'score': 0.36933514,
  'index': 3,
  'word': 'Google',
  'start': 9,
  'end': 15}]

In [60]:
classifier("ele se chama Carlos Alberto. Mora em Juiz de Fora. Trabalha no Carrefour. Seu aniversário é dia 20 de outubro.")


[{'entity': 'LABEL_0',
  'score': 0.9958066,
  'index': 1,
  'word': 'ele',
  'start': 0,
  'end': 3},
 {'entity': 'LABEL_0',
  'score': 0.9984763,
  'index': 2,
  'word': 'se',
  'start': 4,
  'end': 6},
 {'entity': 'LABEL_0',
  'score': 0.9970921,
  'index': 3,
  'word': 'chama',
  'start': 7,
  'end': 12},
 {'entity': 'LABEL_1',
  'score': 0.63242877,
  'index': 4,
  'word': 'Carlos',
  'start': 13,
  'end': 19},
 {'entity': 'LABEL_2',
  'score': 0.60882425,
  'index': 5,
  'word': 'Alberto',
  'start': 20,
  'end': 27},
 {'entity': 'LABEL_0',
  'score': 0.9982097,
  'index': 6,
  'word': '.',
  'start': 27,
  'end': 28},
 {'entity': 'LABEL_0',
  'score': 0.99890673,
  'index': 7,
  'word': 'Mora',
  'start': 29,
  'end': 33},
 {'entity': 'LABEL_0',
  'score': 0.99446625,
  'index': 8,
  'word': 'em',
  'start': 34,
  'end': 36},
 {'entity': 'LABEL_5',
  'score': 0.9924959,
  'index': 9,
  'word': 'Juiz',
  'start': 37,
  'end': 41},
 {'entity': 'LABEL_6',
  'score': 0.98829865,
  '